## Connect to Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/drive/', force_remount=True)
os.chdir('/drive/My Drive/Afstuderen/')

In [ ]:
import scipy as sc
import numpy as np
import pandas as pd
import datetime as dt
import pandas_gbq
import math

## Load data from drive

In [ ]:
# chexpert = pd.read_csv("./cxr/mimic-cxr-2.0.0-chexpert.csv")
metadata = pd.read_csv("./cxr/mimic-cxr-2.0.0-metadata.csv")
# negbio = pd.read_csv("./cxr/mimic-cxr-2.0.0-negbio.csv")


In [ ]:
metadata.head()

In [ ]:
chexpert.head()

In [ ]:
negbio.head()

## Connect to Bigquery

In [ ]:
from google.cloud import bigquery
%load_ext google.cloud.bigquery

pandas_gbq.context.project = 'mimic-data-master-thesis'
pandas_gbq.context.dialect = 'standard'

In [ ]:
from google.colab import auth

auth.authenticate_user()
client = bigquery.Client(project='mimic-data-master-thesis')

In [ ]:
first_subs = metadata['subject_id'].to_list()
first_subs = list(dict.fromkeys(first_subs))

len(first_subs)

In [ ]:
# query_cap = 40
query_cap = 10000

In [ ]:
query = '''
SELECT 
       CE.subject_id, CE.hadm_id, CE.stay_id, CE.charttime, CE.storetime,
       CE.itemid, CE.value, CE.valuenum, CE.valueuom, CE.warning, IT.label,
       IT.abbreviation, IT.linksto, IT.category, IT.param_type, 
       IT.lownormalvalue, IT.highnormalvalue
FROM 
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN `physionet-data.mimic_icu.d_items` AS IT ON CE.itemid = IT.itemid
INNER JOIN `physionet-data.mimic_ed.edstays` AS ED ON CE.subject_id = ED.subject_id AND CE.hadm_id = ED.hadm_id
WHERE 
  CE.subject_id IN UNNEST(@subjects)
  
ORDER BY charttime, storetime, CE.subject_id, stay_id
'''

job_config = bigquery.QueryJobConfig(
    query_parameters = [
      bigquery.ArrayQueryParameter('subjects','NUMERIC', first_subs[0:query_cap] )
    ]
)

query_job = client.query(query, job_config=job_config)

query_job

In [ ]:
query = '''
SELECT 
       CE.subject_id, CE.hadm_id, CE.stay_id, CE.charttime, CE.storetime,
       CE.itemid, CE.value, CE.valuenum, CE.valueuom, CE.warning, IT.label,
       IT.abbreviation, IT.linksto, IT.category, IT.param_type, 
       IT.lownormalvalue, IT.highnormalvalue
FROM 
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN `physionet-data.mimic_icu.d_items` AS IT ON CE.itemid = IT.itemid
INNER JOIN `physionet-data.mimic_ed.edstays` AS ED ON CE.subject_id = ED.subject_id AND CE.hadm_id = ED.hadm_id
WHERE 
  CE.subject_id IN UNNEST(@subjects) AND CE.itemid IN (SELECT itemid FROM `physionet-data.mimic_icu.chartevents` GROUP BY itemid ORDER BY count(itemid) DESC LIMIT 20)
  
ORDER BY charttime, storetime, CE.subject_id, stay_id
'''

job_config = bigquery.QueryJobConfig(
    query_parameters = [
      bigquery.ArrayQueryParameter('subjects','NUMERIC', first_subs[0:query_cap] )
    ]
)

query_job = client.query(query, job_config=job_config)

query_job

In [ ]:
query = '''
SELECT
  CE.subject_id,
  CE.hadm_id,
  CE.stay_id,
  CE.charttime,
  CE.storetime,
  CE.itemid,
  CE.value,
  CE.valuenum,
  CE.valueuom,
  CE.warning,
  IT.label,
  IT.abbreviation,
  IT.linksto,
  IT.category,
  IT.param_type,
  IT.lownormalvalue,
  IT.highnormalvalue
FROM
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN
  `physionet-data.mimic_icu.d_items` AS IT
ON
  CE.itemid = IT.itemid
INNER JOIN
  `physionet-data.mimic_ed.edstays` AS ED
ON
  CE.subject_id = ED.subject_id
  AND CE.hadm_id = ED.hadm_id
WHERE
  CE.hadm_id IN (
  SELECT
    hadm_id 
  FROM (
    SELECT
      hadm_id,
      COUNT(itemid) AS cnt
    FROM
      `physionet-data.mimic_icu.chartevents`
    WHERE
      itemid IN (
      SELECT
        itemid
      FROM
        `physionet-data.mimic_icu.chartevents`
      GROUP BY
        itemid
      ORDER BY
        COUNT(itemid) DESC
      LIMIT
        20)
    GROUP BY
      hadm_id
    ORDER BY
      COUNT(itemid) DESC
    LIMIT
      10000)
  WHERE
    cnt > 3200
    AND CNT <= 3600)
ORDER BY
  charttime,
  storetime,
  CE.subject_id,
  stay_id
'''

job_config = bigquery.QueryJobConfig(
    
)

query_job = client.query(query, job_config=job_config)

query_job

In [ ]:
query = '''
SELECT
  CE.subject_id,
  CE.hadm_id,
  CE.stay_id,
  CE.charttime,
  CE.storetime,
  CE.itemid,
  CE.value,
  CE.valuenum,
  CE.valueuom,
  CE.warning,
  IT.label,
  IT.abbreviation,
  IT.linksto,
  IT.category,
  IT.param_type,
  IT.lownormalvalue,
  IT.highnormalvalue
FROM
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN
  `physionet-data.mimic_icu.d_items` AS IT
ON
  CE.itemid = IT.itemid
INNER JOIN
  `physionet-data.mimic_ed.edstays` AS ED
ON
  CE.subject_id = ED.subject_id
  AND CE.hadm_id = ED.hadm_id
WHERE
  CE.hadm_id IN (
  SELECT
    hadm_id
  FROM (
    SELECT
      hadm_id,
      COUNT(itemid) AS cnt
    FROM
      `physionet-data.mimic_icu.chartevents`
    WHERE
      itemid IN (
      SELECT
        itemid
      FROM
        `physionet-data.mimic_icu.chartevents`
      GROUP BY
        itemid
      ORDER BY
        COUNT(itemid) DESC
      LIMIT
        20)
    GROUP BY
      hadm_id
    ORDER BY
      COUNT(itemid) DESC )
  LIMIT
    4000 )
  AND CE.itemid IN (
  SELECT
    itemid
  FROM
    `physionet-data.mimic_icu.chartevents`
  GROUP BY
    itemid
  ORDER BY
    COUNT(itemid) DESC
  LIMIT
    20)
ORDER BY
  charttime,
  storetime,
  CE.subject_id,
  stay_id
'''

job_config = bigquery.QueryJobConfig(
    
)

query_job = client.query(query, job_config=job_config)

query_job

In [ ]:
query = '''
SELECT
  CE.subject_id,
  CE.hadm_id,
  CE.charttime,
  CE.value,
  CE.valuenum,
  CE.valueuom,
  IT.label,
  IT.category,
  IT.param_type,
FROM
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN
  `physionet-data.mimic_icu.d_items` AS IT
ON
  CE.itemid = IT.itemid
INNER JOIN
  `physionet-data.mimic_ed.edstays` AS ED
ON
  CE.subject_id = ED.subject_id
  AND CE.hadm_id = ED.hadm_id
WHERE
  CE.hadm_id IN (
  SELECT
    hadm_id
  FROM (
    SELECT
      hadm_id,
      COUNT(itemid) AS cnt
    FROM
      `physionet-data.mimic_icu.chartevents`
    WHERE
      itemid IN (
      SELECT
        itemid
      FROM
        `physionet-data.mimic_icu.chartevents`
      GROUP BY
        itemid
      ORDER BY
        COUNT(itemid) DESC
      LIMIT
        20)
    GROUP BY
      hadm_id
    ORDER BY
      COUNT(itemid) DESC )
    LIMIT 10000 )
  AND CE.itemid IN (
  SELECT
    itemid
  FROM
    `physionet-data.mimic_icu.chartevents`
  GROUP BY
    itemid
  ORDER BY
    COUNT(itemid) DESC
  LIMIT
    20)
    ORDER BY
  charttime,
  storetime,
  CE.subject_id

'''

In [ ]:
query = '''
SELECT
  CE.subject_id,
  CE.hadm_id,
  CE.charttime,
  CE.value,
  CE.valuenum,
  CE.valueuom,
  IT.label,
  IT.category,
  IT.param_type,
FROM
  `physionet-data.mimic_icu.chartevents` AS CE
INNER JOIN
  `physionet-data.mimic_icu.d_items` AS IT
ON
  CE.itemid = IT.itemid
WHERE
  CE.hadm_id IN (
  SELECT
    hadm_id
  FROM (
    SELECT
      hadm_id,
      COUNT(itemid) AS cnt
    FROM
      `physionet-data.mimic_icu.chartevents`
    WHERE
      itemid IN (
      SELECT
        itemid
      FROM
        `physionet-data.mimic_icu.chartevents`
      GROUP BY
        itemid
      ORDER BY
        COUNT(itemid) DESC
      LIMIT
        20)
    GROUP BY
      hadm_id
    ORDER BY
      COUNT(itemid) DESC )
    LIMIT 5000 )
  AND CE.itemid IN (
  SELECT
    itemid
  FROM
    `physionet-data.mimic_icu.chartevents`
  GROUP BY
    itemid
  ORDER BY
    COUNT(itemid) DESC
  LIMIT
    20)
    ORDER BY
  charttime,
  storetime,
  CE.subject_id


'''

In [ ]:
job_config = bigquery.QueryJobConfig()

query_job = client.query(query, job_config=job_config)

query_job

In [ ]:
results = query_job.result().to_dataframe(progress_bar_type='tqdm')
results

### Store data as CSV

In [ ]:
results.to_csv('icu_data_5000.csv', index=False)

In [ ]:
results.to_csv(f'first_{query_cap}_patients.csv', index=False)

## Check data

In [ ]:
len(list(results['hadm_id'].unique()))

### Load raw data as CSV

In [ ]:
results = pd.read_csv('first_6000_patients.csv', index_col=0)
results.head()

### Get number of unique labels in ICU

In [ ]:
measurements = results['label'].values.ravel()
unique_measurements = pd.unique(measurements)
len(unique_measurements)

In [ ]:
selected_patients = []
for i in first_subs[0:query_cap]:
  res = len(results[results['subject_id'] == i])
  if (res > 0):
    selected_patients.append(i)

### Identify stays per patient

In [ ]:
num_seq = 0 
max_seq = 0
min_seq = 10000000
min_stay_duration = dt.timedelta(100000, 0, 0)
max_stay_duration = dt.timedelta(0,0,0)
max_stay_duration_id = 0
min_stay_duration_id = 0 
total_stay_duration = 0
min_stay_duration_id = 0
max_stay_duration_id = 0
all_stays = []


for patient in selected_patients:
  stays = results[results['subject_id'] == patient]
  stays_ids = stays['stay_id'].values.ravel()
  unique_stays = pd.unique(stays_ids)
  min_stay = dt.timedelta(100000, 0, 0)
  max_stay = dt.timedelta(0,0,0)
  begin_stay = dt.datetime.now()
  end_stay = dt.datetime.now()
  
  print(f'Unique stays for patient {patient}: {len(unique_stays)}')

  for stay in unique_stays:
    all_stays.append(stay)
    stay_records = results[results['stay_id'] == stay]
    num_records = len(stay_records)
    begin_stay = stay_records['charttime'].min()
    end_stay = stay_records['charttime'].max()
    duration = end_stay - begin_stay
    print(f'  - #records for stay {stay}: {num_records}, stay from {begin_stay} till {end_stay}, duration: {duration}')

    max_stay_duration_id = stay if duration > max_stay_duration else max_stay_duration_id
    min_stay_duration_id = stay if duration < min_stay_duration else min_stay_duration_id
    max_stay_duration = duration if duration > max_stay_duration else max_stay_duration
    min_stay_duration = duration if duration < min_stay_duration else min_stay_duration
    
    max_seq = num_records if num_records > max_seq else max_seq
    min_seq = num_records if num_records < min_seq else min_seq

  if len(unique_stays) > 1:
    total_rec = len(results[results['subject_id'] == patient])
    print('  ---------------------------------------')
    print(f'  Total #records: {total_rec}')

  num_seq += len(unique_stays)

  print()

print(f'Max sequence length: {max_seq}')
print(f'Min sequence length: {min_seq}')
print(f'Average sequence length: {len(results) / num_seq}')
print(f'Total number of sequences: {num_seq}')
print(f'Average number of sequences per patient: {num_seq / len(selected_patients)}')
print(f'Min duration: {min_stay_duration} for stay: {min_stay_duration_id}')
print(f'Max duration: {max_stay_duration} for stay: {max_stay_duration_id}')

## Link stays with scans

### Add charttime to metadata

In [ ]:
metadata['charttime'] = pd.to_datetime(metadata['StudyDate'], format='%Y%m%d')
metadata = metadata.drop(columns=['Rows', 'Columns'])

### Find the stays per patient for each scan

In [ ]:
# Fetch all stays in the ED followed by an ICU
def get_stays(subject_id):
  query = f'SELECT * FROM `physionet-data.mimic_core.transfers` WHERE subject_id = {subject_id} AND (careunit LIKE "%ICU%" OR careunit LIKE "%Emergency Department%" OR careunit IS null) ORDER BY intime, outtime'
  return client.query(query).result().to_dataframe()

In [ ]:
photo_stays = []
for patient in selected_patients:
  images = metadata[metadata['subject_id'] == patient]
  
  patient_stays = get_stays(patient)
  patient_photo_stays = {
      "patient": patient,
      "scans": []
  };

  for image_index, image in images.iterrows():
    stay = 0;
    for index, event in patient_stays.iterrows():
      if event.eventtype == 'ED' and event.intime.date() <= image.charttime.date() and event.outtime.date() >= image.charttime.date() and not event.hadm_id is None:
        stay = event.hadm_id
      elif event.eventtype == 'discharge':
        stay = 0
      elif 'ICU' in str(event.careunit) and not (stay == 0 or math.isnan(stay)):
        patient_photo_stays['scans'].append({
            "scan" : image.study_id,
            "stay": stay
        });
        break
  photo_stays.append(patient_photo_stays)

### Transform photo stay data to DataFrame and store as CSV

In [ ]:
patient_scans_df = pd.DataFrame.from_dict(photo_stays, orient="columns")
patient_scans_df.to_csv('patient_scans.csv')

### Load photo stay data from CSV

In [ ]:
patient_scans_df = pd.read_csv("patient_scans.csv", index_col=0)
patient_scans_df.head()

photo_stays = {}
photo_stays = patient_scans_df.values.tolist()

In [ ]:
import ast

# Get all hospital stays from
hospital_stays = []
scan_id = []

for patient in photo_stays:
  for scan in ast.literal_eval(patient[1]):
    hospital_stays.append(scan['stay'])
    scan_id.append(scan['scan'])
  
 
 
# for patient in photo_stays:
#   for scan in patient['scans']:
#     scan_id.append(scan['scan'])
#     hospital_stays.append(scan['stay'])

hospital_stays = list(dict.fromkeys(hospital_stays))
scan_id = list(dict.fromkeys(scan_id))

# Create final Dataset

In [ ]:
# Get all complete stays for the selected patients 
query = '''
SELECT
  *
FROM
  `physionet-data.mimic_core.transfers`
WHERE
  hadm_id IN UNNEST(@subjects) 
ORDER BY
  intime,
  hadm_id
'''

job_config = bigquery.QueryJobConfig(
    query_parameters = [
      bigquery.ArrayQueryParameter('subjects', 'NUMERIC', results['hadm_id'].to_numpy())
    ]
)
# job_config = bigquery.QueryJobConfig(
#     query_parameters = [
#       bigquery.ArrayQueryParameter('subjects', 'NUMERIC', results['hadm_id'].to_numpy() )
#     ]
# )
all_stays = client.query(query, job_config=job_config).result().to_dataframe()
all_stays

### Store all stays as CSV

In [ ]:
all_stays.to_csv("all_stays.csv", index=False)

### Read all stays from CSV

In [ ]:
all_stays = pd.read_csv("all_stays.csv", index_col=0)

## Concatenate all datasets

In [ ]:
all_stays['charttime'] = pd.to_datetime(all_stays['intime'])
results['charttime'] = pd.to_datetime(results['charttime'], format='%Y-%m-%d')

In [ ]:
ed_icu = pd.concat([metadata[metadata['study_id'].isin(scan_id)], results])

In [ ]:
admission_ed_icu = pd.concat([ed_icu, all_stays])

In [ ]:
for col in admission_ed_icu.columns:
    print(col)

In [ ]:
admission_ed_icu.sort_values(by="charttime")

In [ ]:
admission_ed_icu.to_csv('complete_data_v1.7.csv')

In [ ]:
full_data = pd.read_csv('complete_data_v1.7.csv')

# Create subset for development purposes

# Obtain metadata

In [ ]:
admissions['stay_duration'] =  admissions['dischtime'] - admissions['admittime']
admissions.describe()

In [ ]:
data = pd.read_csv('complete_data_v1.csv')

In [ ]:
for col in data.columns:
    print(col)

In [ ]:
for col in metadata.columns:
  print(col)

print("\n ------- \n")

for col in results.columns:
  print(col)

print("\n ------- \n")

for col in all_stays.columns:
  print(col)

In [ ]:
selected_patient_scans = metadata[metadata['study_id'].isin(scan_id)]

In [ ]:
print(f'# Photos per patient: {selected_patient_scans.groupby(["study_id"]).size()}')
print(f'Avg. # photos per patient: {selected_patient_scans.groupby(["study_id"]).size().mean()}')



In [ ]:
len(selected_patient_scans)

In [ ]:
all_stays.groupby(['subject_id']).size()